In [61]:
import torch
from torch.utils.cpp_extension import load
from algo import original_orth
import timeit

In [94]:
qr_func = load(name="qr_func", sources=["qr_orthogonalization.cpp", "qr_orthogonalization.cu"])

In [95]:
M = 8
N = 1024
device = torch.device("cuda")

A = torch.rand((M, N), device=device)
Q = torch.zeros((M,N), device=device)

In [96]:
qr_func.qr_orthogonalization(A.clone(), Q, 0)

In [97]:
Q @ Q.T

tensor([[ 1.0000e+00,  4.6566e-09, -5.5879e-09,  1.1176e-08,  3.7253e-09,
         -1.0245e-08,  2.3283e-09,  1.3970e-09],
        [ 4.6566e-09,  1.0000e+00, -1.2922e-08, -5.1223e-09, -5.5879e-09,
          2.7940e-09,  1.8626e-09,  1.3039e-08],
        [-5.5879e-09, -1.2922e-08,  1.0000e+00,  4.6566e-10, -9.3132e-10,
         -1.8626e-09, -9.3132e-10,  2.0955e-09],
        [ 1.1176e-08, -5.1223e-09,  4.6566e-10,  1.0000e+00, -2.7940e-09,
          6.5193e-09, -9.3132e-10,  3.7253e-09],
        [ 3.7253e-09, -5.5879e-09, -9.3132e-10, -2.7940e-09,  1.0000e+00,
         -7.9162e-09, -5.5879e-09, -5.5879e-09],
        [-1.0245e-08,  2.7940e-09, -1.8626e-09,  6.5193e-09, -7.9162e-09,
          1.0000e+00,  1.4901e-08,  9.3132e-09],
        [ 2.3283e-09,  1.8626e-09, -9.3132e-10, -9.3132e-10, -5.5879e-09,
          1.4901e-08,  1.0000e+00,  3.7253e-09],
        [ 1.3970e-09,  1.3039e-08,  2.0955e-09,  3.7253e-09, -5.5879e-09,
          9.3132e-09,  3.7253e-09,  1.0000e+00]], device='cuda:0'

In [99]:
torch.norm(Q @ Q.T - torch.eye(M, device = device))

tensor(6.3555e-07, device='cuda:0')

In [100]:
Q_or = original_orth(A.clone(), 0)

In [101]:
torch.norm(Q - Q_or)

tensor(0.2016, device='cuda:0')

In [102]:
torch.norm(Q_or @ Q_or.T - torch.eye(M, device = device))

tensor(6.7086e-07, device='cuda:0')

In [103]:
%timeit torch.cuda.synchronize(); qr_func.qr_orthogonalization(A.clone(), Q, 0); torch.cuda.synchronize();

418 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [104]:
%timeit torch.cuda.synchronize(); torch.qr(A.clone()); torch.cuda.synchronize();

10.2 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
